In [37]:
import numpy as np
import scipy.optimize as opt
import pandas as pd
import os
try:
    import cPickle as pickle
except ImportError:
    import pickle
from ogusa import wealth
from ogusa import labor
from ogusa import SS
from ogusa import utils

from taxcalc import Calculator
from dask.distributed import Client
from ogusa.parameters import Specifications
from ogusa.utils import REFORM_DIR, BASELINE_DIR

In [38]:
output_base = BASELINE_DIR
client = Client(processes=False)
num_workers = 1  # multiprocessing.cpu_count()

alpha_T = np.zeros(50)
alpha_T[0:2] = 0.09
alpha_T[2:10] = 0.09 + 0.01
alpha_T[10:40] = 0.09 - 0.01
alpha_T[40:] = 0.09
alpha_G = np.zeros(7)
alpha_G[0:3] = 0.05 - 0.01
alpha_G[3:6] = 0.05 - 0.005
alpha_G[6:] = 0.05
small_open = False

#ref = Calculator.read_json_param_objects('2017_law.json', None)
#reform = ref['policy']


user_params = {'frisch': 0.41, 'start_year': 2018,
                   'tau_b': [(0.21 * 0.55) * (0.017 / 0.055), (0.21 * 0.55) * (0.017 / 0.055)],
                   'debt_ratio_ss': 1.0, 'alpha_T': alpha_T.tolist(),
                   'alpha_G': alpha_G.tolist(), 'small_open': small_open}

kwargs = {'output_base': output_base, 'baseline_dir': BASELINE_DIR,
              'test': False, 'time_path': False, 'baseline': True,
              'user_params': user_params, 'guid': '_example',
              'run_micro': False, 'data': 'cps', 'client': client,
              'num_workers': num_workers}

p = Specifications(run_micro=False, output_base=output_base,
                          baseline_dir=BASELINE_DIR, test=False,
                          time_path=False, baseline=True,
                          #reform=reform, 
                           guid='_example', data='cps',
                          client=client, num_workers=num_workers)

[2.36e-05, 0.00013240000000000002, 0.0006012, 0.002038, 0.004459199999999999, 0.008911199999999998, 0.0158764, 0.023537999999999993, 0.0318616, 0.041342, 0.053616000000000004, 0.06809200000000001, 0.0838056, 0.09883760000000004, 0.11027560000000003, 0.11716800000000001, 0.11726519999999997, 0.1106636, 0.10044399999999999, 0.08805239999999998, 0.0751132, 0.06239799999999999, 0.05025399999999999, 0.0385556, 0.028371999999999994, 0.020465200000000003, 0.014030399999999998, 0.00891, 0.005257600000000001, 0.0028715999999999998, 0.0014236000000000001, 0.0006244, 0.0002456, 9.48e-05, 3.960000000000001e-05, 2.1200000000000004e-05, 2e-05]
         Year   Age    Values
12099  1995.0  14.0  0.000012
12100  1995.0  15.0  0.000066
12101  1995.0  16.0  0.000301
12102  1995.0  17.0  0.001019
12103  1995.0  18.0  0.002230
12104  1995.0  19.0  0.004456
12105  1995.0  20.0  0.007938
12106  1995.0  21.0  0.011769
12107  1995.0  22.0  0.015931
12108  1995.0  23.0  0.020671
12109  1995.0  24.0  0.026808
12

/Users/keiirizawa/Desktop/OG-USA/ogusa/demographics.py:400: RuntimeWarning: divide by zero encountered in double_scalars
  pow = end_surv / tot_period_surv


New periods:  100
Orig periods:  100
New periods:  100
Orig periods:  100
New periods:  100
Orig periods:  100
New periods:  100
Orig periods:  100
[2.36e-05, 0.00013240000000000002, 0.0006012, 0.002038, 0.004459199999999999, 0.008911199999999998, 0.0158764, 0.023537999999999993, 0.0318616, 0.041342, 0.053616000000000004, 0.06809200000000001, 0.0838056, 0.09883760000000004, 0.11027560000000003, 0.11716800000000001, 0.11726519999999997, 0.1106636, 0.10044399999999999, 0.08805239999999998, 0.0751132, 0.06239799999999999, 0.05025399999999999, 0.0385556, 0.028371999999999994, 0.020465200000000003, 0.014030399999999998, 0.00891, 0.005257600000000001, 0.0028715999999999998, 0.0014236000000000001, 0.0006244, 0.0002456, 9.48e-05, 3.960000000000001e-05, 2.1200000000000004e-05, 2e-05]
         Year   Age    Values
12099  1995.0  14.0  0.000012
12100  1995.0  15.0  0.000066
12101  1995.0  16.0  0.000301
12102  1995.0  17.0  0.001019
12103  1995.0  18.0  0.002230
12104  1995.0  19.0  0.004456
1210

In [39]:
def chi_n_func(s, a0, a1, a2, a3, a4):
    chi_n = a0 + a1*s + a2*s**2 + a3*s**3 + a4*s**4
    return chi_n

In [44]:
a0 = 1
a1 = 0
a2 = 0
a3 = 0
a4 = 0

params_init = np.array([a0, a1, a2, a3, a4])

labor_data = np.array([167, 165, 165, 165, 165, 166, 165, 165, 164, 166, 164])
labor_moments = labor_data * 12 / (365 * 17.5)
data_moments = np.array(list(labor_moments.flatten()))
ages = np.linspace(20, 100, p.S)
p.chi_n = chi_n_func(ages, a0, a1, a2, a3, a4)
### had to add this to make it work:
ss_output = SS.run_SS(p, client)
model_moments = calc_moments(ss_output, p.omega_SS, p.lambdas, p.S, p.J)

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((<function apply at 0x1a1fa37400>, <function fsolve at 0x118d942f0>, [<function euler_equation_solver at 0x1a1fa206a8>, array([0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063,
       0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063,
       0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063,
       0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063,
       0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063,
       0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063,
       0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063,
       0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063,
       0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.36 ,
       0.36 , 0.36 , 0.36 , 0.36 , 0.36 , 0.36 , 0.36 , 0.36 , 0.36 ,
       0.36 , 0.36 , 0.36 , 0.36 , 0.36 , 0.36 , 0.36 , 0.36 , 0.36 ,
       0.36 , 0.36 , 0.36 ,

AttributeError: 'Specifications' object has no attribute 'etr_params'

tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x1a20c2efd0> exception was never retrieved: Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/anaconda3/lib/python3.6/site-packages/distributed/client.py", line 1357, in wait
    raise AllExit()
distributed.client.AllExit
tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x1a20dac860> exception was never retrieved: Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1069, in run
    yielded = self.gen.send(value)
  File "/anaconda3/lib/python3.6/site-packages/distributed/client.py", line 1357, in wait
    raise AllExit()
distributed.client.AllExit
tornado.application - ERROR - Future <tornado.concurrent.Future object at 0x1a20dac908> exception was never retrieved: Traceback (most recent call last):
  File "/anaconda3/lib/python3